In [1]:
info = """
LangChain 1.0 - Middleware Basics (中间件基础)
==============================================

本模块重点讲解：
1. 什么是中间件（Middleware）
2. before_model 和 after_model 钩子
3. 自定义中间件的创建
4. 多个中间件的组合
5. 内置中间件的使用
"""
print(info)


LangChain 1.0 - Middleware Basics (中间件基础)

本模块重点讲解：
1. 什么是中间件（Middleware）
2. before_model 和 after_model 钩子
3. 自定义中间件的创建
4. 多个中间件的组合
5. 内置中间件的使用



In [2]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain.agents.middleware import AgentMiddleware
from langgraph.checkpoint.memory import InMemorySaver


In [3]:
# 加载环境变量
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY or GROQ_API_KEY == "your_groq_api_key_here":
    raise ValueError(
        "\n请先在 .env 文件中设置有效的 GROQ_API_KEY\n"
        "访问 https://console.groq.com/keys 获取免费密钥"
    )

# 初始化模型
model = init_chat_model("groq:llama-3.3-70b-versatile", api_key=GROQ_API_KEY)



In [4]:
@tool
def get_weather(city: str) -> str:
    """查询城市天气"""
    weather_data = {
        "北京": "晴天，15°C",
        "上海": "多云，18°C",
        "深圳": "雨天，22°C"
    }
    return weather_data.get(city, "未知城市")


In [5]:
print("""
  关键规则

  1. 必须继承 AgentMiddleware ← 这个固定
  2. 方法名固定 (before_model, after_model) ← 这个固定
  3. 类名随意 ← 这个不固定

  LangGraph 只看：
  - 是否继承 AgentMiddleware？
  - 是否有 before_model / after_model 方法？
  """)


  关键规则

  1. 必须继承 AgentMiddleware ← 这个固定
  2. 方法名固定 (before_model, after_model) ← 这个固定
  3. 类名随意 ← 这个不固定

  LangGraph 只看：
  - 是否继承 AgentMiddleware？
  - 是否有 before_model / after_model 方法？
  


In [17]:
class LoggingMiddleware(AgentMiddleware): #类名任意
    """
    日志中间件 - 记录每次模型调用

    before_model: 模型调用前执行
    after_model: 模型响应后执行
    """
    def before_model(self, state, runtime):
        """模型调用前"""
        print("\n[中间件] before_model: 准备调用模型")
        print(f"[中间件] 当前消息数: {len(state.get('messages', []))}")
        print(f"state: {state}")
        print(f"runtime: {runtime}")
        return None  # 返回 None 表示继续正常流程
    def after_model(self, state, runtime):
        """模型响应后"""
        print("[中间件] after_model: 模型已响应")
        last_message = state.get('messages', [])[-1]
        print(f"[中间件] 响应类型: {last_message.__class__.__name__}")
        print(f"state: {state}")
        return None  # 返回 None 表示不修改状态

In [18]:
print( """
    示例1：基础中间件 - 日志记录

    展示 before_model 和 after_model 的基本用法
    """)


    示例1：基础中间件 - 日志记录

    展示 before_model 和 after_model 的基本用法
    


In [19]:
print("\n" + "="*70)
print("示例 1：基础中间件 - 日志记录")
print("="*70)



示例 1：基础中间件 - 日志记录


In [20]:
agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。",
    middleware=[LoggingMiddleware()]  # 添加中间件
)

In [21]:
response = agent.invoke({
    "messages":[
        {"role": "user", "content": "你好"}
    ]
})
print(response)


[中间件] before_model: 准备调用模型
[中间件] 当前消息数: 1
state: {'messages': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}, id='ee19dcff-7974-4c36-9429-decb8e0ea14a')]}
runtime: Runtime(context=None, store=None, stream_writer=<function Pregel.stream.<locals>.stream_writer at 0x1110da7a0>, previous=None)
[中间件] after_model: 模型已响应
[中间件] 响应类型: AIMessage
state: {'messages': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}, id='ee19dcff-7974-4c36-9429-decb8e0ea14a'), AIMessage(content='你好。今天我能为你做些什么？', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 45, 'total_tokens': 58, 'completion_time': 0.081659925, 'completion_tokens_details': None, 'prompt_time': 0.001969592, 'prompt_tokens_details': None, 'queue_time': 0.052921608, 'total_time': 0.083629517}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_pr

In [22]:
print("\n关键点：")
print("  - before_model 在模型调用前执行")
print("  - after_model 在模型响应后执行")
print("  - 返回 None 表示不修改状态")


关键点：
  - before_model 在模型调用前执行
  - after_model 在模型响应后执行
  - 返回 None 表示不修改状态


In [23]:
class CallCounterMiddleware(AgentMiddleware):
    """
    调用计数中间件 - 统计模型调用次数
    """
    def __init__(self):
        self.call_count = 0

    def before_model(self, state, runtime):
        self.call_count += 1
        print(f"\n[中间件] 模型调用次数: {self.call_count}")
        return None

In [24]:
print( """
    示例2：中间件内部状态 - 计数器

    展示如何在中间件内部维护状态（不依赖 Agent state）
    """)


    示例2：中间件内部状态 - 计数器

    展示如何在中间件内部维护状态（不依赖 Agent state）
    


In [25]:
print("\n" + "="*70)
print("示例 2：中间件内部状态 - 模型调用计数")
print("="*70)


示例 2：中间件内部状态 - 模型调用计数


In [41]:
agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。",
    middleware=[CallCounterMiddleware()],
    checkpointer=InMemorySaver()
)


In [42]:
config = {"configurable": {"thread_id": "counter_test"}}


In [43]:
print("\n第一次调用:")
agent.invoke({"messages": [{"role": "user", "content": "你好"}]}, config)


第一次调用:

[中间件] 模型调用次数: 1


{'messages': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}, id='1f2a87ed-e2b2-4f28-85c6-46c6d6aa0aa4'),
  AIMessage(content='你好！我可以如何帮助你今天？你有任何问题，或者需要帮助完成某项任务，或者只是想聊天？', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 45, 'total_tokens': 75, 'completion_time': 0.174341693, 'completion_tokens_details': None, 'prompt_time': 0.007224189, 'prompt_tokens_details': None, 'queue_time': 0.052992671, 'total_time': 0.181565882}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3f8c-d7da-7630-b373-3ffc58041d52-0', usage_metadata={'input_tokens': 45, 'output_tokens': 30, 'total_tokens': 75})]}

In [44]:
print("\n第二次调用:")
agent.invoke({"messages": [{"role": "user", "content": "今天天气"}]}, config)



第二次调用:

[中间件] 模型调用次数: 2


{'messages': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}, id='1f2a87ed-e2b2-4f28-85c6-46c6d6aa0aa4'),
  AIMessage(content='你好！我可以如何帮助你今天？你有任何问题，或者需要帮助完成某项任务，或者只是想聊天？', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 45, 'total_tokens': 75, 'completion_time': 0.174341693, 'completion_tokens_details': None, 'prompt_time': 0.007224189, 'prompt_tokens_details': None, 'queue_time': 0.052992671, 'total_time': 0.181565882}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3f8c-d7da-7630-b373-3ffc58041d52-0', usage_metadata={'input_tokens': 45, 'output_tokens': 30, 'total_tokens': 75}),
  HumanMessage(content='今天天气', additional_kwargs={}, response_metadata={}, id='7c6c2dac-a96a-4714-b357-ca9a45e81649'),
  AIMessage(content='我很高兴你问了！不过，我是一个大型语言模型，我没有实时访问当前位置的天气信息。但是，我可以建议一些方

In [45]:
print("\n第三次调用:")
response = agent.invoke({"messages": [{"role": "user", "content": "谢谢"}]}, config)
response



第三次调用:

[中间件] 模型调用次数: 3


{'messages': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}, id='1f2a87ed-e2b2-4f28-85c6-46c6d6aa0aa4'),
  AIMessage(content='你好！我可以如何帮助你今天？你有任何问题，或者需要帮助完成某项任务，或者只是想聊天？', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 45, 'total_tokens': 75, 'completion_time': 0.174341693, 'completion_tokens_details': None, 'prompt_time': 0.007224189, 'prompt_tokens_details': None, 'queue_time': 0.052992671, 'total_time': 0.181565882}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3f8c-d7da-7630-b373-3ffc58041d52-0', usage_metadata={'input_tokens': 45, 'output_tokens': 30, 'total_tokens': 75}),
  HumanMessage(content='今天天气', additional_kwargs={}, response_metadata={}, id='7c6c2dac-a96a-4714-b357-ca9a45e81649'),
  AIMessage(content='我很高兴你问了！不过，我是一个大型语言模型，我没有实时访问当前位置的天气信息。但是，我可以建议一些方

In [46]:
print("\n关键点：")
print("  - 中间件内部维护计数器（self.count）")
print("  - 不依赖 Agent state（更可靠）")
print("  - 返回 None 表示不修改 Agent 状态")


关键点：
  - 中间件内部维护计数器（self.count）
  - 不依赖 Agent state（更可靠）
  - 返回 None 表示不修改 Agent 状态


In [47]:
class MessageTrimmerMiddleware(AgentMiddleware):
    """
    消息修剪中间件 - 限制消息数量

    before_model 修改消息列表
    注意：需要配合无 checkpointer 使用，否则历史会被恢复
    """

    def __init__(self, max_messages=5):
        super().__init__()
        self.max_messages = max_messages
        self.trimmed_count = 0  # 统计修剪次数

    def before_model(self, state, runtime):
        """模型调用前，修剪消息"""
        messages = state.get('messages', [])

        if len(messages) > self.max_messages:
            # 保留最近的 N 条消息
            trimmed_messages = messages[-self.max_messages:]
            self.trimmed_count += 1
            print(f"\n[修剪] 消息从 {len(messages)} 条减少到 {len(trimmed_messages)} 条 (第{self.trimmed_count}次修剪)")
            return {"messages": trimmed_messages}

        return None

In [49]:
print( """
    示例3：消息修剪 - 防止消息过多

    展示如何在调用前修改消息列表
    重点：手动累积消息，观察修剪效果
    """)


    示例3：消息修剪 - 防止消息过多

    展示如何在调用前修改消息列表
    重点：手动累积消息，观察修剪效果
    


In [50]:
print("\n" + "="*70)
print("示例 3：消息修剪 - 限制消息数量")
print("="*70)

print("\n[说明] 不使用 checkpointer，手动管理消息历史\n")


示例 3：消息修剪 - 限制消息数量

[说明] 不使用 checkpointer，手动管理消息历史



In [51]:
middleware = MessageTrimmerMiddleware(max_messages=4)  # 最多保留 4 条
agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。",
    middleware=[middleware]
    # 不使用 checkpointer
)

In [52]:
messages = []
for i in range(6):
    print(f"\n--- 第 {i+1} 次对话 ---")

    # 新增用户消息
    new_msg = {"role": "user", "content": f"消息{i+1}：简短回复"}
    messages.append(new_msg)

    print(f"调用前消息数: {len(messages)}")

    # 调用 agent（middleware会修剪）
    response = agent.invoke({"messages": messages})

    # 获取完整对话（包含AI响应）
    messages = response['messages']

    print(f"调用后消息数: {len(messages)}")
    if len(messages) <= 4:
        print(f"消息列表: {[m.content[:15] for m in messages]}")

print(f"\n修剪统计: 共修剪了 {middleware.trimmed_count} 次")


--- 第 1 次对话 ---
调用前消息数: 1
调用后消息数: 2
消息列表: ['消息1：简短回复', '我能帮你什么？']

--- 第 2 次对话 ---
调用前消息数: 3
调用后消息数: 4
消息列表: ['消息1：简短回复', '我能帮你什么？', '消息2：简短回复', '您今天需要什么帮助？']

--- 第 3 次对话 ---
调用前消息数: 5

[修剪] 消息从 5 条减少到 4 条 (第1次修剪)
调用后消息数: 6

--- 第 4 次对话 ---
调用前消息数: 7

[修剪] 消息从 7 条减少到 4 条 (第2次修剪)
调用后消息数: 8

--- 第 5 次对话 ---
调用前消息数: 9

[修剪] 消息从 9 条减少到 4 条 (第3次修剪)
调用后消息数: 10

--- 第 6 次对话 ---
调用前消息数: 11

[修剪] 消息从 11 条减少到 4 条 (第4次修剪)
调用后消息数: 12

修剪统计: 共修剪了 4 次


In [53]:
print("\n关键点：")
print("  - before_model 在传给模型前修剪消息")
print("  - max_messages=4 限制发送给模型的消息数")
print("  - 但返回的 response 会包含新生成的消息")
print("  - 不使用 checkpointer 避免历史恢复")
print("\n生产建议：")
print("  - 简单修剪用这种方式")
print("  - 复杂场景用 SummarizationMiddleware（第8章）")


关键点：
  - before_model 在传给模型前修剪消息
  - max_messages=4 限制发送给模型的消息数
  - 但返回的 response 会包含新生成的消息
  - 不使用 checkpointer 避免历史恢复

生产建议：
  - 简单修剪用这种方式
  - 复杂场景用 SummarizationMiddleware（第8章）


In [54]:
class OutputValidationMiddleware(AgentMiddleware):
    """
    输出验证中间件 - 检查响应长度

    after_model 验证输出
    """
    def __init__(self,max_length=100):
        super().__init__()
        self.max_length = max_length
    def after_model(self,state,runtime):
        '''模型响应后，验证输出'''
        messages = state.get('messages', [])
        if not messages:
            return None
        last_message = messages[-1]
        content = getattr(last_message, 'content', '')
        if len(content) > self.max_length:
            print(f"\n[警告] 响应过长 ({len(content)} 字符)，已截断到 {self.max_length}")
        return None


In [55]:
print( """
    示例4：输出验证 - 检查响应质量

    展示如何验证模型输出
    """)


    示例4：输出验证 - 检查响应质量

    展示如何验证模型输出
    


In [56]:
print("\n" + "="*70)
print("示例 4：输出验证 - 响应长度检查")
print("="*70)



示例 4：输出验证 - 响应长度检查


In [57]:
agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。",
    middleware=[OutputValidationMiddleware(max_length=50)]
)

In [59]:
response = agent.invoke({
    "messages": [{"role": "user", "content": "请详细介绍 Python 编程语言的历史、特点和应用"}]
})
response["messages"][-1].content



[警告] 响应过长 (1544 字符)，已截断到 50


'Python是一种高级、解释型编程语言，具有简单、易读和易用的特点。以下是Python的详细介绍：\n\n**历史**\n\nPython由荷兰程序员Guido van Rossum在20世纪80年代末开始设计和开发。 Guido van Rossum当时在荷兰的国家数学和计算机科学研究所（CWI）工作，他希望创建一种能够快速开发应用程序的语言。 1989年，Guido van Rossum开始编写Python解释器的代码，并于1991年发布了Python 0.9.1版本。之后，Python逐渐获得了广泛的关注和支持，成为一种流行的编程语言。\n\n**特点**\n\nPython具有以下特点：\n\n1. **简单易学**：Python的语法简单明了，易于学习和使用，即使对于初学者也是如此。\n2. **高级语言**：Python是一种高级语言，它提供了许多内置的数据类型和函数，能够简化编程的过程。\n3. **解释型语言**：Python是一种解释型语言，它不需要编译，能够直接运行代码。\n4. **动态类型**：Python是一种动态类型语言，它不需要明确地指定变量的类型。\n5. **面向对象**：Python是一种面向对象的语言，它支持类、对象和继承等概念。\n6. **跨平台**：Python能够在多种操作系统上运行，包括Windows、Linux和macOS。\n\n**应用**\n\nPython在以下领域有广泛的应用：\n\n1. **Web开发**：Python可以用于Web开发，例如使用Django和Flask等框架来创建Web应用程序。\n2. **数据分析**：Python可以用于数据分析，例如使用Pandas和NumPy等库来处理和分析数据。\n3. **机器学习**：Python可以用于机器学习，例如使用TensorFlow和Keras等库来创建机器学习模型。\n4. **自动化**：Python可以用于自动化，例如使用PyAutoGUI和Selenium等库来自动化GUI和Web应用程序。\n5. **科学计算**：Python可以用于科学计算，例如使用SciPy和Matplotlib等库来进行科学计算和数据可视化。\n6. **游戏开发**：Python可以用于游戏开发，例如使用Pygame和Panda3D等库来创建游戏。\n\n

In [60]:
print("\n关键点：")
print("  - after_model 可以验证输出")
print("  - 可以实现重试、截断等逻辑")
print("  - 保证输出质量")


关键点：
  - after_model 可以验证输出
  - 可以实现重试、截断等逻辑
  - 保证输出质量


In [61]:
class TimingMiddleware(AgentMiddleware):
    """计时中间件"""

    def before_model(self, state, runtime):
        import time
        # 记录开始时间（实际应该用 runtime 的上下文管理）
        print("\n[计时] 开始调用模型...")
        return None

    def after_model(self, state, runtime):
        print("[计时] 模型调用完成")
        return None

In [62]:
print("""
    示例5：多个中间件 - 执行顺序

    展示中间件的执行顺序：
    - before_model: 正序（1→2→3）
    - after_model: 逆序（3→2→1）
    """)


    示例5：多个中间件 - 执行顺序

    展示中间件的执行顺序：
    - before_model: 正序（1→2→3）
    - after_model: 逆序（3→2→1）
    


In [63]:
print("\n" + "="*70)
print("示例 5：多个中间件 - 执行顺序")
print("="*70)


示例 5：多个中间件 - 执行顺序


In [64]:
class Middleware1(AgentMiddleware):
    def before_model(self, state, runtime):
        print("[中间件1] before_model")
        return None
    def after_model(self, state, runtime):
        print("[中间件1] after_model")
        return None

class Middleware2(AgentMiddleware):
    def before_model(self, state, runtime):
        print("[中间件2] before_model")
        return None
    def after_model(self, state, runtime):
        print("[中间件2] after_model")
        return None

class Middleware3(AgentMiddleware):
    def before_model(self, state, runtime):
        print("[中间件3] before_model")
        return None
    def after_model(self, state, runtime):
        print("[中间件3] after_model")
        return None

In [65]:
agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。",
    middleware=[Middleware1(), Middleware2(), Middleware3()]
)

In [66]:
agent.invoke({
    "messages": [{"role": "user", "content": "测试多个中间件的执行顺序"}]
})

[中间件1] before_model
[中间件2] before_model
[中间件3] before_model
[中间件3] after_model
[中间件2] after_model
[中间件1] after_model


{'messages': [HumanMessage(content='测试多个中间件的执行顺序', additional_kwargs={}, response_metadata={}, id='bbce5a92-a3a6-4f1e-bd77-987f960917b4'),
  AIMessage(content='一个非常好的测试用例！\n\n在许多 Web 框架中，中间件（Middleware）是用于处理请求和响应的函数或类，它们可以被串联起来执行。测试多个中间件的执行顺序是一个非常重要的测试场景。\n\n以下是一个测试多个中间件的执行顺序的例子：\n```python\nimport unittest\nfrom your_framework import Middleware, Request, Response\n\n# 中间件 1：记录请求时间\nclass Middleware1(Middleware):\n    def process_request(self, request):\n        request.start_time = datetime.now()\n        return request\n\n    def process_response(self, request, response):\n        print(f"Middleware 1: Request took {datetime.now() - request.start_time} seconds")\n        return response\n\n# 中间件 2：检查用户身份\nclass Middleware2(Middleware):\n    def process_request(self, request):\n        if not request.user.is_authenticated:\n            raise Exception("User is not authenticated")\n        return request\n\n    def process_response(self, request, response):\n        print(f"Middleware 

In [67]:
print("\n关键点：")
print("  - before_model: 正序执行（1→2→3）")
print("  - after_model: 逆序执行（3→2→1）")
print("  - 类似洋葱模型：1→2→3→模型→3→2→1")


关键点：
  - before_model: 正序执行（1→2→3）
  - after_model: 逆序执行（3→2→1）
  - 类似洋葱模型：1→2→3→模型→3→2→1


In [68]:
class MaxCallsMiddleware(AgentMiddleware):
    """
    最大调用限制中间件

    通过抛出异常来阻止模型调用（更可靠的方式）
    """

    def __init__(self, max_calls=3):
        super().__init__()
        self.max_calls = max_calls
        self.count = 0  # 简单计数器

    def before_model(self, state, runtime):
        """检查调用次数，超过限制则抛出异常"""
        if self.count >= self.max_calls:
            print(f"\n[限制] 已达到最大调用次数 {self.max_calls}，停止调用")
            # 抛出自定义异常来阻止继续执行
            raise ValueError(f"已达到最大调用次数限制: {self.max_calls}")

        print(f"[限制] 当前调用次数: {self.count}/{self.max_calls}")
        return None

    def after_model(self, state, runtime):
        """增加计数"""
        self.count += 1
        print("次数+1")
        return None

In [69]:
print( """
    示例6：调用限制 - 通过异常阻止调用

    展示如何使用异常来阻止模型调用（比 jump_to 更可靠）
    """)


    示例6：调用限制 - 通过异常阻止调用

    展示如何使用异常来阻止模型调用（比 jump_to 更可靠）
    


In [70]:
print("\n" + "="*70)
print("示例 6：调用限制 - 最大调用次数")
print("="*70)


示例 6：调用限制 - 最大调用次数


In [71]:

agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。",
    middleware=[MaxCallsMiddleware(max_calls=2)],
    checkpointer=InMemorySaver()
)

In [72]:
config = {"configurable": {"thread_id": "limit_test"}}

for i in range(4):
    print(f"\n--- 第 {i+1} 次尝试调用 ---")
    try:
        response = agent.invoke(
            {"messages": [{"role": "user", "content": f"消息{i+1}"}]},
            config
        )
        if response.get('messages'):
            print(f"响应: {response['messages'][-1].content[:50]}...")
    except ValueError as e:
        print(f"[已阻止] {e}")
    except Exception as e:
        print(f"调用失败: {e}")


--- 第 1 次尝试调用 ---
[限制] 当前调用次数: 0/2
次数+1
响应: 你好！我很高兴你联系了我。有什么可以我帮忙的吗？...

--- 第 2 次尝试调用 ---
[限制] 当前调用次数: 1/2
次数+1
响应: 我感觉你今天的信息有点简短。想聊点什么，还是需要一些帮助？我全神贯注，随时准备！...

--- 第 3 次尝试调用 ---

[限制] 已达到最大调用次数 2，停止调用
[已阻止] 已达到最大调用次数限制: 2

--- 第 4 次尝试调用 ---

[限制] 已达到最大调用次数 2，停止调用
[已阻止] 已达到最大调用次数限制: 2


In [73]:
print("\n关键点：")
print("  - before_model 中抛出异常可以阻止模型调用")
print("  - 比 jump_to 更可靠（在 LangChain 1.0 中）")
print("  - 中间件内部维护计数（self.count）")
print("  - 用于实现熔断、限流等逻辑")
print("\n注意：")
print("  - jump_to 在 middleware 中可能不按预期工作")
print("  - 推荐用异常来实现流程控制")


关键点：
  - before_model 中抛出异常可以阻止模型调用
  - 比 jump_to 更可靠（在 LangChain 1.0 中）
  - 中间件内部维护计数（self.count）
  - 用于实现熔断、限流等逻辑

注意：
  - jump_to 在 middleware 中可能不按预期工作
  - 推荐用异常来实现流程控制


In [74]:
print("""
    示例7：内置中间件 - SummarizationMiddleware

    使用 LangChain 提供的内置中间件
    """)


    示例7：内置中间件 - SummarizationMiddleware

    使用 LangChain 提供的内置中间件
    


In [75]:
print("\n" + "="*70)
print("示例 7：内置中间件 - 自动摘要")
print("="*70)


示例 7：内置中间件 - 自动摘要


In [76]:
from langchain.agents.middleware import SummarizationMiddleware

In [77]:
agent = create_agent(
    model=model,
    tools=[],
    system_prompt="你是一个有帮助的助手。",
    middleware=[
        SummarizationMiddleware(
            model="groq:llama-3.3-70b-versatile",
            max_tokens_before_summary=200  # 超过 200 token 就摘要
        )
    ],
    checkpointer=InMemorySaver()
)

/var/folders/2t/w_0f5ts90870qhccyfzjmdgh0000gn/T/ipykernel_56363/1531878392.py:6: DeprecationWarning: max_tokens_before_summary is deprecated. Use trigger=('tokens', value) instead.
  SummarizationMiddleware(


In [78]:
config = {"configurable": {"thread_id": "summary_test"}}

# 连续多轮对话
conversations = [
    "介绍一下 Python",
    "它有哪些特点？",
    "主要应用在哪些领域？",
    "和 Java 的区别是什么？"
]

In [79]:
for i, msg in enumerate(conversations):
    print(f"\n--- 第 {i+1} 轮对话 ---")
    print(f"用户: {msg}")
    response = agent.invoke({"messages": [{"role": "user", "content": msg}]}, config)
    print(f"Agent: {response['messages'][-1].content[:100]}...")
    print(f"总消息数: {len(response['messages'])}")


--- 第 1 轮对话 ---
用户: 介绍一下 Python
Agent: **Python 编程语言介绍**

Python 是一种高级、解释型的编程语言，于 1991 年由 Guido van Rossum 首次发布。...
总消息数: 2

--- 第 2 轮对话 ---
用户: 它有哪些特点？
Agent: **Python 的特性**

Python 编程语言具有以下特点：

**一般特性**
----------------

1. **易于学习**：Pyth...
总消息数: 4

--- 第 3 轮对话 ---
用户: 主要应用在哪些领域？
Agent: **Python 的主要应用领域**

Python 编程语言在多个领域有着广泛的应用。以下是Python的一些主要应用领域：

**1. 网络开...
总消息数: 6

--- 第 4 轮对话 ---
用户: 和 Java 的区别是什么？
Agent: **Python 和 Java 的主要区别**

Python 和 Java 是两种流行的编程语言，具有不同的设计目标、语法和用例。以下是P...
总消息数: 8


In [80]:
print("\n关键点：")
print("  - SummarizationMiddleware 自动摘要旧消息")
print("  - 防止消息历史无限增长")
print("  - 第 08 章详细学习过")


关键点：
  - SummarizationMiddleware 自动摘要旧消息
  - 防止消息历史无限增长
  - 第 08 章详细学习过
